<a href="https://colab.research.google.com/github/RemyaVKarthikeyan/AA-Stagecoach-Project/blob/main/01_07_2024_Fetching_the_time_table_of_a_given_stop_point_of_given_lineID%2C_direction_(based_on_day_of_the_week)_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import requests
from datetime import datetime
import pytz

# Function to fetch data from the API
def fetch_data(url):
    response = requests.get(url)
    return response.json()

# Function to extract schedule names
def extract_schedule_names(data, schedule_names_dict={}):
    if isinstance(data, dict):
        if data.get('$type') == "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities" and 'knownJourneys' in data:
            if 'name' in data:
                schedule_names_dict[data['name']] = data['knownJourneys']
        for key, value in data.items():
            extract_schedule_names(value, schedule_names_dict)
    elif isinstance(data, list):
        for item in data:
            extract_schedule_names(item, schedule_names_dict)
    return schedule_names_dict

# Function to categorize journeys into hourly slots
def categorize_into_slots(timetable):
    slots = [[] for _ in range(24)]
    for journey in timetable:
        hour = int(journey['hour'])  # Convert hour to integer
        if 0 <= hour < 24:  # Ensure hour is within the valid range
            slots[hour].append(journey)
    return slots

# Function to calculate the number of buses between the first time available in the slot and the current time
def calculate_buses(slot, current_hour, current_minute):
    bus_count = 0
    for journey in slot:
        journey_hour = int(journey['hour'])
        journey_minute = int(journey['minute'])
        if journey_hour == current_hour and journey_minute <= current_minute:
            bus_count += 1
    return bus_count

# Main function to get user input and print the schedule names and timetable
def main():
    line_id = input("Enter the line ID: ")
    stop_point_id = input("Enter the stop point ID: ")
    direction = input("Enter the direction (outbound/inbound): ")

    # Construct the URL
    url = f'https://api.tfl.gov.uk/Line/{line_id}/Timetable/{stop_point_id}?direction={direction}'

    # Fetching data
    data = fetch_data(url)

    # Extracting schedule names
    schedule_names_dict = extract_schedule_names(data)

    # Printing the extracted schedule names as a numbered list
    print("Schedule Names:")
    schedule_names = list(schedule_names_dict.keys())
    for i, name in enumerate(schedule_names, start=1):
        print(f"{i}. {name}")

    # Asking the user to select a schedule
    selected_number = int(input("Enter the number corresponding to the schedule name (which matches with the day of the week): "))
    selected_schedule_name = schedule_names[selected_number - 1]

    # Categorizing the timetable into hourly slots
    timetable = schedule_names_dict[selected_schedule_name]
    slots = categorize_into_slots(timetable)

    # Get the current time in BST
    bst = pytz.timezone('Europe/London')
    now = datetime.now(bst)
    current_hour = now.hour
    current_minute = now.minute

    # Calculate the number of buses scheduled between the first time available in the current slot and the current time
    bus_count = calculate_buses(slots[current_hour], current_hour, current_minute)
    print(f"\nNumber of buses scheduled between the first time available in the current slot and now: {bus_count}")

    # Print the timetable slots corresponding to the current hour, the previous hour, and the next hour
    hours_to_print = [current_hour - 1, current_hour, current_hour + 1]
    print("\nTimetable:")
    print("Time   Line ID  Stop Point ID  Direction")
    for hour in hours_to_print:
        if 0 <= hour < 24:
            print(f"\nSlot {hour}:")
            for journey in slots[hour]:
                journey_hour = str(journey['hour']).zfill(2)
                journey_minute = str(journey['minute']).zfill(2)
                time = f"{journey_hour}:{journey_minute}"
                print(f"{time}   {line_id}    {stop_point_id}  {direction.capitalize()}")

# Run the main function
if __name__ == "__main__":
    main()


Enter the line ID: 179
Enter the stop point ID: 490005183E
Enter the direction (outbound/inbound): outbound
Schedule Names:
1. Mon-Fri Non-Schooldays
2. Mon-Fri Schooldays
3. Saturday
4. Sunday
Enter the number corresponding to the schedule name (which matches with the day of the week): 2

Number of buses scheduled between the first time available in the current slot and now: 1

Timetable:
Time   Line ID  Stop Point ID  Direction

Slot 11:
11:10   179    490005183E  Outbound
11:22   179    490005183E  Outbound
11:34   179    490005183E  Outbound
11:46   179    490005183E  Outbound
11:58   179    490005183E  Outbound

Slot 12:
12:10   179    490005183E  Outbound
12:22   179    490005183E  Outbound
12:34   179    490005183E  Outbound
12:46   179    490005183E  Outbound
12:58   179    490005183E  Outbound

Slot 13:
13:10   179    490005183E  Outbound
13:22   179    490005183E  Outbound
13:34   179    490005183E  Outbound
13:46   179    490005183E  Outbound
13:58   179    490005183E  Outbo

In [4]:
import requests
from datetime import datetime
import pytz

# Function to fetch data from the API
def fetch_data(url):
    response = requests.get(url)
    return response.json()

# Function to extract schedule names based on day of the week
def extract_schedule_names(data, day_of_week):
    schedule_names_dict = {}
    if isinstance(data, dict):
        if data.get('$type') == "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities":
            if 'name' in data and day_of_week in data['name']:
                schedule_names_dict[data['name']] = data['knownJourneys']
        for key, value in data.items():
            schedule_names_dict.update(extract_schedule_names(value, day_of_week))
    elif isinstance(data, list):
        for item in data:
            schedule_names_dict.update(extract_schedule_names(item, day_of_week))
    return schedule_names_dict

# Function to categorize journeys into hourly slots
def categorize_into_slots(timetable):
    slots = [[] for _ in range(24)]
    for journey in timetable:
        hour = int(journey['hour'])  # Convert hour to integer
        if 0 <= hour < 24:  # Ensure hour is within the valid range
            slots[hour].append(journey)
    return slots

# Function to calculate the number of buses between the first time available in the slot and the current time
def calculate_buses(slot, current_hour, current_minute):
    bus_count = 0
    for journey in slot:
        journey_hour = int(journey['hour'])
        journey_minute = int(journey['minute'])
        if journey_hour == current_hour and journey_minute <= current_minute:
            bus_count += 1
    return bus_count

# Main function to get user input and print the schedule names and timetable
def main():
    line_id = input("Enter the line ID: ")
    stop_point_id = input("Enter the stop point ID: ")
    direction = input("Enter the direction (outbound/inbound): ")

    # Construct the URL
    url = f'https://api.tfl.gov.uk/Line/{line_id}/Timetable/{stop_point_id}?direction={direction}'

    # Fetching data
    data = fetch_data(url)

    # Get current day of the week in BST
    bst = pytz.timezone('Europe/London')
    now = datetime.now(bst)
    day_of_week = now.strftime('%A')  # Get full day name (e.g., 'Monday')

    # Extracting schedule names for the current day of the week
    schedule_names_dict = extract_schedule_names(data, day_of_week)

    # Printing the extracted schedule names as a numbered list
    print("Schedule Names:")
    schedule_names = list(schedule_names_dict.keys())
    for i, name in enumerate(schedule_names, start=1):
        print(f"{i}. {name}")

    # Automatically select the schedule matching the current day of the week
    selected_schedule_name = None
    for name in schedule_names:
        if day_of_week in name:
            selected_schedule_name = name
            break

    if not selected_schedule_name:
        print(f"No schedule found for {day_of_week}.")
        return

    print(f"\nAutomatically selected schedule for {day_of_week}: {selected_schedule_name}")

    # Categorizing the timetable into hourly slots
    timetable = schedule_names_dict[selected_schedule_name]
    slots = categorize_into_slots(timetable)

    # Get current hour and minute
    current_hour = now.hour
    current_minute = now.minute

    # Calculate the number of buses scheduled between the first time available in the current slot and the current time
    bus_count = calculate_buses(slots[current_hour], current_hour, current_minute)
    print(f"\nNumber of buses scheduled between the first time available in the current slot and now: {bus_count}")

    # Print the timetable slots corresponding to the current hour, the previous hour, and the next hour
    hours_to_print = [current_hour - 1, current_hour, current_hour + 1]
    print("\nTimetable:")
    print("Time   Line ID  Stop Point ID  Direction")
    for hour in hours_to_print:
        if 0 <= hour < 24:
            print(f"\nSlot {hour}:")
            for journey in slots[hour]:
                journey_hour = str(journey['hour']).zfill(2)
                journey_minute = str(journey['minute']).zfill(2)
                time = f"{journey_hour}:{journey_minute}"
                print(f"{time}   {line_id}    {stop_point_id}  {direction.capitalize()}")

# Run the main function
if __name__ == "__main__":
    main()


Enter the line ID: 179
Enter the stop point ID: 490005183E
Enter the direction (outbound/inbound): outbound
Schedule Names:
No schedule found for Monday.


In [5]:
from datetime import datetime
import pytz

# Get current time in BST
bst = pytz.timezone('Europe/London')
now = datetime.now(bst)

# Fetch day of the week
day_of_week = now.strftime('%A')  # %A gives full weekday name (e.g., 'Monday')

print(f"Today is {day_of_week}.")


Today is Monday.


In [6]:
from datetime import datetime
import pytz

# Function to fetch the current day of the week
def get_day_of_week():
    bst = pytz.timezone('Europe/London')
    now = datetime.now(bst)
    return now.strftime('%A')  # %A gives full weekday name (e.g., 'Monday')

# Function to select schedule name based on day of the week
def select_schedule_name(day_of_week):
    schedule_map = {
        'Monday': 'Mon-Fri Schooldays',
        'Tuesday': 'Mon-Fri Schooldays',
        'Wednesday': 'Mon-Fri Schooldays',
        'Thursday': 'Mon-Fri Schooldays',
        'Friday': 'Mon-Fri Schooldays',
        'Saturday': 'Saturday',
        'Sunday': 'Sunday'
    }
    return schedule_map.get(day_of_week, None)

# Example usage
if __name__ == "__main__":
    # Fetch day of the week
    day_of_week = get_day_of_week()
    print(f"Today is {day_of_week}.")

    # Select schedule name based on day of the week
    selected_schedule_name = select_schedule_name(day_of_week)

    if selected_schedule_name:
        print(f"Selected schedule name: {selected_schedule_name}")
    else:
        print("No matching schedule name found.")


Today is Monday.
Selected schedule name: Mon-Fri Schooldays


In [9]:
from datetime import datetime
import pytz

# Function to fetch the current day of the week
def get_day_of_week():
    bst = pytz.timezone('Europe/London')
    now = datetime.now(bst)
    return now.strftime('%A')  # %A gives full weekday name (e.g., 'Monday')

# Function to select schedule name based on day of the week
def select_schedule_name(day_of_week):
    schedule_map = {
        'Monday': 'Monday to Thursday',
        'Tuesday': 'Monday to Thursday',
        'Wednesday': 'Monday to Thursday',
        'Thursday': 'Monday to Thursday',
        'Friday': 'Friday',
        'Saturday': 'Saturday',
        'Sunday': 'Sunday'
    }

    second_schedule_map = {
        'Monday': 'Mon-Fri Schooldays',
        'Tuesday': 'Mon-Fri Schooldays',
        'Wednesday': 'Mon-Fri Schooldays',
        'Thursday': 'Mon-Fri Schooldays',
        'Friday': 'Mon-Fri Schooldays',
        'Saturday': 'Saturday',
        'Sunday': 'Sunday'
    }

    third_schedule_map = {
        'Monday': 'Monday to Friday',
        'Tuesday': 'Monday to Friday',
        'Wednesday': 'Monday to Friday',
        'Thursday': 'Monday to Friday',
        'Friday': 'Monday to Friday',
        'Saturday': 'Saturday',
        'Sunday': 'Sunday'
    }

    # Select the appropriate schedule name based on day of the week
    if day_of_week in schedule_map:
        return schedule_map[day_of_week]
    elif day_of_week in second_schedule_map:
        return second_schedule_map[day_of_week]
    elif day_of_week in third_schedule_map:
        return third_schedule_map[day_of_week]
    else:
        return None

# Example usage
if __name__ == "__main__":
    # Fetch day of the week
    day_of_week = get_day_of_week()
    print(f"Today is {day_of_week}.")

    # Select schedule name based on day of the week
    selected_schedule_name = select_schedule_name(day_of_week)

    if selected_schedule_name:
        print(f"Selected schedule name: {selected_schedule_name}")
    else:
        print("No matching schedule name found.")


Today is Monday.
Selected schedule name: Monday to Thursday


In [11]:
import requests
from datetime import datetime
import pytz

# Function to fetch data from the API
def fetch_data(url):
    response = requests.get(url)
    return response.json()

# Function to extract schedule names
def extract_schedule_names(data, schedule_names_dict={}):
    if isinstance(data, dict):
        if data.get('$type') == "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities" and 'knownJourneys' in data:
            if 'name' in data:
                schedule_names_dict[data['name']] = data['knownJourneys']
        for key, value in data.items():
            extract_schedule_names(value, schedule_names_dict)
    elif isinstance(data, list):
        for item in data:
            extract_schedule_names(item, schedule_names_dict)
    return schedule_names_dict

# Function to categorize journeys into hourly slots
def categorize_into_slots(timetable):
    slots = [[] for _ in range(24)]
    for journey in timetable:
        hour = int(journey['hour'])  # Convert hour to integer
        if 0 <= hour < 24:  # Ensure hour is within the valid range
            slots[hour].append(journey)
    return slots

# Function to calculate the number of buses between the first time available in the slot and the current time
def calculate_buses(slot, current_hour, current_minute):
    bus_count = 0
    for journey in slot:
        journey_hour = int(journey['hour'])
        journey_minute = int(journey['minute'])
        if journey_hour == current_hour and journey_minute <= current_minute:
            bus_count += 1
    return bus_count

# Function to fetch the current day of the week
def get_day_of_week():
    bst = pytz.timezone('Europe/London')
    now = datetime.now(bst)
    return now.strftime('%A')  # %A gives full weekday name (e.g., 'Monday')

# Function to select schedule name based on day of the week
def select_schedule_name(day_of_week):
    schedule_map = {
        'Monday': 'Monday to Thursday',
        'Tuesday': 'Monday to Thursday',
        'Wednesday': 'Monday to Thursday',
        'Thursday': 'Monday to Thursday',
        'Friday': 'Friday',
        'Saturday': 'Saturday',
        'Sunday': 'Sunday'
    }

    second_schedule_map = {
        'Monday': 'Mon-Fri Schooldays',
        'Tuesday': 'Mon-Fri Schooldays',
        'Wednesday': 'Mon-Fri Schooldays',
        'Thursday': 'Mon-Fri Schooldays',
        'Friday': 'Mon-Fri Schooldays',
        'Saturday': 'Saturday',
        'Sunday': 'Sunday'
    }

    third_schedule_map = {
        'Monday': 'Monday to Friday',
        'Tuesday': 'Monday to Friday',
        'Wednesday': 'Monday to Friday',
        'Thursday': 'Monday to Friday',
        'Friday': 'Monday to Friday',
        'Saturday': 'Saturday',
        'Sunday': 'Sunday'
    }

    # Select the appropriate schedule name based on day of the week
    if day_of_week in schedule_map:
        return schedule_map[day_of_week]
    elif day_of_week in second_schedule_map:
        return second_schedule_map[day_of_week]
    elif day_of_week in third_schedule_map:
        return third_schedule_map[day_of_week]
    else:
        return None

# Main function to get user input and print the schedule names and timetable
def main():
    line_id = input("Enter the line ID: ")
    stop_point_id = input("Enter the stop point ID: ")
    direction = input("Enter the direction (outbound/inbound): ")

    # Fetching data
    url = f'https://api.tfl.gov.uk/Line/{line_id}/Timetable/{stop_point_id}?direction={direction}'
    data = fetch_data(url)

    # Extracting schedule names
    schedule_names_dict = extract_schedule_names(data)

    # Fetch day of the week
    day_of_week = get_day_of_week()

    # Select schedule name based on day of the week
    selected_schedule_name = select_schedule_name(day_of_week)

    if selected_schedule_name:
        print(f"Today is {day_of_week}.")
        print(f"Selected schedule name: {selected_schedule_name}")

        # Categorizing the timetable into hourly slots
        timetable = schedule_names_dict[selected_schedule_name]
        slots = categorize_into_slots(timetable)

        # Get the current time in BST
        bst = pytz.timezone('Europe/London')
        now = datetime.now(bst)
        current_hour = now.hour
        current_minute = now.minute

        # Calculate the number of buses scheduled between the first time available in the current slot and the current time
        bus_count = calculate_buses(slots[current_hour], current_hour, current_minute)
        print(f"\nNumber of buses scheduled between the first time available in the current slot and now: {bus_count}")

        # Print the timetable slots corresponding to the current hour, the previous hour, and the next hour
        hours_to_print = [current_hour - 1, current_hour, current_hour + 1]
        print("\nTimetable:")
        print("Time   Line ID  Stop Point ID  Direction")
        for hour in hours_to_print:
            if 0 <= hour < 24:
                print(f"\nSlot {hour}:")
                for journey in slots[hour]:
                    journey_hour = str(journey['hour']).zfill(2)
                    journey_minute = str(journey['minute']).zfill(2)
                    time = f"{journey_hour}:{journey_minute}"
                    print(f"{time}   {line_id}    {stop_point_id}  {direction.capitalize()}")

    else:
        print("No matching schedule name found.")

# Run the main function
if __name__ == "__main__":
    main()


Enter the line ID: 179
Enter the stop point ID: 490005183E
Enter the direction (outbound/inbound): outbound
Today is Monday.
Selected schedule name: Monday to Thursday


KeyError: 'Monday to Thursday'

In [13]:
import requests
from datetime import datetime
import pytz

# Function to fetch data from the API
def fetch_data(url):
    response = requests.get(url)
    return response.json()

# Function to extract schedule names
def extract_schedule_names(data, schedule_names_dict={}):
    if isinstance(data, dict):
        if data.get('$type') == "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities" and 'knownJourneys' in data:
            if 'name' in data:
                schedule_names_dict[data['name']] = data['knownJourneys']
        for key, value in data.items():
            extract_schedule_names(value, schedule_names_dict)
    elif isinstance(data, list):
        for item in data:
            extract_schedule_names(item, schedule_names_dict)
    return schedule_names_dict

# Function to categorize journeys into hourly slots
def categorize_into_slots(timetable):
    slots = [[] for _ in range(24)]
    for journey in timetable:
        hour = int(journey['hour'])  # Convert hour to integer
        if 0 <= hour < 24:  # Ensure hour is within the valid range
            slots[hour].append(journey)
    return slots

# Function to calculate the number of buses between the first time available in the slot and the current time
def calculate_buses(slot, current_hour, current_minute):
    bus_count = 0
    for journey in slot:
        journey_hour = int(journey['hour'])
        journey_minute = int(journey['minute'])
        if journey_hour == current_hour and journey_minute <= current_minute:
            bus_count += 1
    return bus_count

# Function to fetch the current day of the week
def get_day_of_week():
    bst = pytz.timezone('Europe/London')
    now = datetime.now(bst)
    return now.strftime('%A')  # %A gives full weekday name (e.g., 'Monday')

# Main function to get user input and print the schedule names and timetable
def main():
    line_id = input("Enter the line ID: ")
    stop_point_id = input("Enter the stop point ID: ")
    direction = input("Enter the direction (outbound/inbound): ")

    # Fetching data
    url = f'https://api.tfl.gov.uk/Line/{line_id}/Timetable/{stop_point_id}?direction={direction}'
    data = fetch_data(url)

    # Extracting schedule names
    schedule_names_dict = extract_schedule_names(data)

    # Fetch day of the week
    day_of_week = get_day_of_week()
    print(f"Today is {day_of_week}.")

    # Print extracted schedule names
    print("\nExtracted Schedule Names:")
    for i, schedule_name in enumerate(schedule_names_dict.keys(), start=1):
        print(f"{i}. {schedule_name}")

    # Determine the matching schedule name based on the current day of the week
    selected_schedule_name = None
    for schedule_name in schedule_names_dict.keys():
        if day_of_week.lower() in schedule_name.lower():
            selected_schedule_name = schedule_name
            break

    if selected_schedule_name:
        print(f"\nSelected schedule name: {selected_schedule_name}")

        # Categorizing the timetable into hourly slots
        timetable = schedule_names_dict[selected_schedule_name]
        slots = categorize_into_slots(timetable)

        # Get the current time in BST
        bst = pytz.timezone('Europe/London')
        now = datetime.now(bst)
        current_hour = now.hour
        current_minute = now.minute

        # Calculate the number of buses scheduled between the first time available in the current slot and the current time
        bus_count = calculate_buses(slots[current_hour], current_hour, current_minute)
        print(f"\nNumber of buses scheduled between the first time available in the current slot and now: {bus_count}")

        # Print the timetable slots corresponding to the current hour, the previous hour, and the next hour
        hours_to_print = [current_hour - 1, current_hour, current_hour + 1]
        print("\nTimetable:")
        print("Time   Line ID  Stop Point ID  Direction")
        for hour in hours_to_print:
            if 0 <= hour < 24:
                print(f"\nSlot {hour}:")
                for journey in slots[hour]:
                    journey_hour = str(journey['hour']).zfill(2)
                    journey_minute = str(journey['minute']).zfill(2)
                    time = f"{journey_hour}:{journey_minute}"
                    print(f"{time}   {line_id}    {stop_point_id}  {direction.capitalize()}")

    else:
        print("No matching schedule name found for today.")

# Run the main function
if __name__ == "__main__":
    main()


Enter the line ID: 179
Enter the stop point ID: 490005183E
Enter the direction (outbound/inbound): outbound
Today is Monday.

Extracted Schedule Names:
1. Mon-Fri Non-Schooldays
2. Mon-Fri Schooldays
3. Saturday
4. Sunday
No matching schedule name found for today.


In [16]:
import requests
from datetime import datetime
import pytz

# Function to fetch data from the API
def fetch_data(url):
    response = requests.get(url)
    return response.json()

# Function to extract schedule names
def extract_schedule_names(data, schedule_names_dict={}):
    if isinstance(data, dict):
        if data.get('$type') == "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities" and 'knownJourneys' in data:
            if 'name' in data:
                schedule_names_dict[data['name']] = data['knownJourneys']
        for key, value in data.items():
            extract_schedule_names(value, schedule_names_dict)
    elif isinstance(data, list):
        for item in data:
            extract_schedule_names(item, schedule_names_dict)
    return schedule_names_dict

# Function to categorize journeys into hourly slots
def categorize_into_slots(timetable):
    slots = [[] for _ in range(24)]
    for journey in timetable:
        hour = int(journey['hour'])  # Convert hour to integer
        if 0 <= hour < 24:  # Ensure hour is within the valid range
            slots[hour].append(journey)
    return slots

# Function to calculate the number of buses between the first time available in the slot and the current time
def calculate_buses(slot, current_hour, current_minute):
    bus_count = 0
    for journey in slot:
        journey_hour = int(journey['hour'])
        journey_minute = int(journey['minute'])
        if journey_hour == current_hour and journey_minute <= current_minute:
            bus_count += 1
    return bus_count

# Function to fetch the current day of the week
def get_day_of_week():
    bst = pytz.timezone('Europe/London')
    now = datetime.now(bst)
    return now.strftime('%A')  # %A gives full weekday name (e.g., 'Monday')

# Main function to get user input and print the schedule names and timetable
def main():
    line_id = input("Enter the line ID: ")
    stop_point_id = input("Enter the stop point ID: ")
    direction = input("Enter the direction (outbound/inbound): ")

    # Fetching data
    url = f'https://api.tfl.gov.uk/Line/{line_id}/Timetable/{stop_point_id}?direction={direction}'
    data = fetch_data(url)

    # Extracting schedule names
    schedule_names_dict = extract_schedule_names(data)

    # Fetch day of the week
    day_of_week = get_day_of_week()
    print(f"Today is {day_of_week}.")

    # Print extracted schedule names
    print("\nExtracted Schedule Names:")
    for i, schedule_name in enumerate(schedule_names_dict.keys(), start=1):
        print(f"{i}. {schedule_name}")

    # Determine the matching schedule name based on the current day of the week
    selected_schedule_name = None

    if day_of_week.lower() == 'monday':
        # Check for the preferred schedule names for Monday
        preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Thursday', 'Monday to Friday']
        for preferred_name in preferred_schedule_names:
            if preferred_name in schedule_names_dict:
                selected_schedule_name = preferred_name
                break
    else:
        # For other days of the week, select based on exact match
        for schedule_name in schedule_names_dict.keys():
            if day_of_week.lower() in schedule_name.lower():
                selected_schedule_name = schedule_name
                break

    if selected_schedule_name:
        print(f"\nSelected schedule name: {selected_schedule_name}")

        # Categorizing the timetable into hourly slots
        timetable = schedule_names_dict[selected_schedule_name]
        slots = categorize_into_slots(timetable)

        # Get the current time in BST
        bst = pytz.timezone('Europe/London')
        now = datetime.now(bst)
        current_hour = now.hour
        current_minute = now.minute

        # Calculate the number of buses scheduled between the first time available in the current slot and the current time
        bus_count = calculate_buses(slots[current_hour], current_hour, current_minute)
        print(f"\nNumber of buses scheduled between the first time available in the current slot and now: {bus_count}")

        # Print the timetable slots corresponding to the current hour, the previous hour, and the next hour
        hours_to_print = [current_hour - 1, current_hour, current_hour + 1]
        print("\nTimetable:")
        print("Time   Line ID  Stop Point ID  Direction")
        for hour in hours_to_print:
            if 0 <= hour < 24:
                print(f"\nSlot {hour}:")
                for journey in slots[hour]:
                    journey_hour = str(journey['hour']).zfill(2)
                    journey_minute = str(journey['minute']).zfill(2)
                    time = f"{journey_hour}:{journey_minute}"
                    print(f"{time}   {line_id}    {stop_point_id}  {direction.capitalize()}")

    else:
        print("No matching schedule name found for today.")

# Run the main function
if __name__ == "__main__":
    main()


Enter the line ID: d7
Enter the stop point ID: 490000038F
Enter the direction (outbound/inbound): inbound
Today is Monday.

Extracted Schedule Names:
1. Monday to Friday
2. Saturday
3. Sunday

Selected schedule name: Monday to Friday

Number of buses scheduled between the first time available in the current slot and now: 3

Timetable:
Time   Line ID  Stop Point ID  Direction

Slot 11:
11:09   d7    490000038F  Inbound
11:21   d7    490000038F  Inbound
11:33   d7    490000038F  Inbound
11:45   d7    490000038F  Inbound
11:57   d7    490000038F  Inbound

Slot 12:
12:09   d7    490000038F  Inbound
12:21   d7    490000038F  Inbound
12:33   d7    490000038F  Inbound
12:45   d7    490000038F  Inbound
12:57   d7    490000038F  Inbound

Slot 13:
13:09   d7    490000038F  Inbound
13:21   d7    490000038F  Inbound
13:33   d7    490000038F  Inbound
13:45   d7    490000038F  Inbound
13:57   d7    490000038F  Inbound


In [ ]:
Monday - Tuesday

In [ ]:
import requests
from datetime import datetime
import pytz

# Function to fetch data from the API
def fetch_data(url):
    response = requests.get(url)
    return response.json()

# Function to extract schedule names
def extract_schedule_names(data, schedule_names_dict={}):
    if isinstance(data, dict):
        if data.get('$type') == "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities" and 'knownJourneys' in data:
            if 'name' in data:
                schedule_names_dict[data['name']] = data['knownJourneys']
        for key, value in data.items():
            extract_schedule_names(value, schedule_names_dict)
    elif isinstance(data, list):
        for item in data:
            extract_schedule_names(item, schedule_names_dict)
    return schedule_names_dict

# Function to categorize journeys into hourly slots
def categorize_into_slots(timetable):
    slots = [[] for _ in range(24)]
    for journey in timetable:
        hour = int(journey['hour'])  # Convert hour to integer
        if 0 <= hour < 24:  # Ensure hour is within the valid range
            slots[hour].append(journey)
    return slots

# Function to calculate the number of buses between the first time available in the slot and the current time
def calculate_buses(slot, current_hour, current_minute):
    bus_count = 0
    for journey in slot:
        journey_hour = int(journey['hour'])
        journey_minute = int(journey['minute'])
        if journey_hour == current_hour and journey_minute <= current_minute:
            bus_count += 1
    return bus_count

# Function to fetch the current day of the week
def get_day_of_week():
    bst = pytz.timezone('Europe/London')
    now = datetime.now(bst)
    return now.strftime('%A')  # %A gives full weekday name (e.g., 'Monday')

# Main function to get user input and print the schedule names and timetable
def main():
    line_id = input("Enter the line ID: ")
    stop_point_id = input("Enter the stop point ID: ")
    direction = input("Enter the direction (outbound/inbound): ")

    # Fetching data
    url = f'https://api.tfl.gov.uk/Line/{line_id}/Timetable/{stop_point_id}?direction={direction}'
    data = fetch_data(url)

    # Extracting schedule names
    schedule_names_dict = extract_schedule_names(data)

    # Fetch day of the week
    day_of_week = get_day_of_week()
    print(f"Today is {day_of_week}.")

    # Print extracted schedule names
    print("\nExtracted Schedule Names:")
    for i, schedule_name in enumerate(schedule_names_dict.keys(), start=1):
        print(f"{i}. {schedule_name}")

    # Determine the matching schedule name based on the current day of the week
    selected_schedule_name = None

    if day_of_week.lower() == 'monday' or day_of_week.lower() == 'tuesday':
        # Check for the preferred schedule names for Monday and Tuesday
        preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Thursday', 'Monday to Friday']
        for preferred_name in preferred_schedule_names:
            if preferred_name in schedule_names_dict:
                selected_schedule_name = preferred_name
                break
    else:
        # For other days of the week, select based on exact match
        for schedule_name in schedule_names_dict.keys():
            if day_of_week.lower() in schedule_name.lower():
                selected_schedule_name = schedule_name
                break

    if selected_schedule_name:
        print(f"\nSelected schedule name: {selected_schedule_name}")

        # Categorizing the timetable into hourly slots
        timetable = schedule_names_dict[selected_schedule_name]
        slots = categorize_into_slots(timetable)

        # Get the current time in BST
        bst = pytz.timezone('Europe/London')
        now = datetime.now(bst)
        current_hour = now.hour
        current_minute = now.minute

        # Calculate the number of buses scheduled between the first time available in the current slot and the current time
        bus_count = calculate_buses(slots[current_hour], current_hour, current_minute)
        print(f"\nNumber of buses scheduled between the first time available in the current slot and now: {bus_count}")

        # Print the timetable slots corresponding to the current hour, the previous hour, and the next hour
        hours_to_print = [current_hour - 1, current_hour, current_hour + 1]
        print("\nTimetable:")
        print("Time   Line ID  Stop Point ID  Direction")
        for hour in hours_to_print:
            if 0 <= hour < 24:
                print(f"\nSlot {hour}:")
                for journey in slots[hour]:
                    journey_hour = str(journey['hour']).zfill(2)
                    journey_minute = str(journey['minute']).zfill(2)
                    time = f"{journey_hour}:{journey_minute}"
                    print(f"{time}   {line_id}    {stop_point_id}  {direction.capitalize()}")

    else:
        print("No matching schedule name found for today.")

# Run the main function
if __name__ == "__main__":
    main()


Trial for Tuesday


In [20]:
import requests
from datetime import datetime
import pytz

# Function to fetch data from the API
def fetch_data(url):
    response = requests.get(url)
    return response.json()

# Function to extract schedule names
def extract_schedule_names(data, schedule_names_dict={}):
    if isinstance(data, dict):
        if data.get('$type') == "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities" and 'knownJourneys' in data:
            if 'name' in data:
                schedule_names_dict[data['name']] = data['knownJourneys']
        for key, value in data.items():
            extract_schedule_names(value, schedule_names_dict)
    elif isinstance(data, list):
        for item in data:
            extract_schedule_names(item, schedule_names_dict)
    return schedule_names_dict

# Function to categorize journeys into hourly slots
def categorize_into_slots(timetable):
    slots = [[] for _ in range(24)]
    for journey in timetable:
        hour = int(journey['hour'])  # Convert hour to integer
        if 0 <= hour < 24:  # Ensure hour is within the valid range
            slots[hour].append(journey)
    return slots

# Function to calculate the number of buses between the first time available in the slot and the current time
def calculate_buses(slot, current_hour, current_minute):
    bus_count = 0
    for journey in slot:
        journey_hour = int(journey['hour'])
        journey_minute = int(journey['minute'])
        if journey_hour == current_hour and journey_minute <= current_minute:
            bus_count += 1
    return bus_count

# Function to fetch the current day of the week (for testing purposes, override to simulate Tuesday)
def get_day_of_week():
    return 'Tuesday'

# Main function to get user input and print the schedule names and timetable
def main():
    line_id = input("Enter the line ID: ")
    stop_point_id = input("Enter the stop point ID: ")
    direction = input("Enter the direction (outbound/inbound): ")

    # Fetching data
    url = f'https://api.tfl.gov.uk/Line/{line_id}/Timetable/{stop_point_id}?direction={direction}'
    data = fetch_data(url)

    # Extracting schedule names
    schedule_names_dict = extract_schedule_names(data)

    # Fetch day of the week
    day_of_week = get_day_of_week()
    print(f"Today is {day_of_week}.")

    # Print extracted schedule names
    print("\nExtracted Schedule Names:")
    for i, schedule_name in enumerate(schedule_names_dict.keys(), start=1):
        print(f"{i}. {schedule_name}")

    # Determine the matching schedule name based on the current day of the week
    selected_schedule_name = None

    if day_of_week.lower() == 'monday' or day_of_week.lower() == 'tuesday':
        # Check for the preferred schedule names for Monday and Tuesday
        preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Thursday', 'Monday to Friday']
        for preferred_name in preferred_schedule_names:
            if preferred_name in schedule_names_dict:
                selected_schedule_name = preferred_name
                break
    else:
        # For other days of the week, select based on exact match
        for schedule_name in schedule_names_dict.keys():
            if day_of_week.lower() in schedule_name.lower():
                selected_schedule_name = schedule_name
                break

    if selected_schedule_name:
        print(f"\nSelected schedule name: {selected_schedule_name}")

        # Categorizing the timetable into hourly slots
        timetable = schedule_names_dict[selected_schedule_name]
        slots = categorize_into_slots(timetable)

        # Get the current time in BST
        bst = pytz.timezone('Europe/London')
        now = datetime.now(bst)
        current_hour = now.hour
        current_minute = now.minute

        # Calculate the number of buses scheduled between the first time available in the current slot and the current time
        bus_count = calculate_buses(slots[current_hour], current_hour, current_minute)
        print(f"\nNumber of buses scheduled between the first time available in the current slot and now: {bus_count}")

        # Print the timetable slots corresponding to the current hour, the previous hour, and the next hour
        hours_to_print = [current_hour - 1, current_hour, current_hour + 1]
        print("\nTimetable:")
        print("Time   Line ID  Stop Point ID  Direction")
        for hour in hours_to_print:
            if 0 <= hour < 24:
                print(f"\nSlot {hour}:")
                for journey in slots[hour]:
                    journey_hour = str(journey['hour']).zfill(2)
                    journey_minute = str(journey['minute']).zfill(2)
                    time = f"{journey_hour}:{journey_minute}"
                    print(f"{time}   {line_id}    {stop_point_id}  {direction.capitalize()}")

    else:
        print("No matching schedule name found for today.")

# Run the main function
if __name__ == "__main__":
    main()


Enter the line ID: d7
Enter the stop point ID: 490000038F
Enter the direction (outbound/inbound): inbound
Today is Tuesday.

Extracted Schedule Names:
1. Monday to Friday
2. Saturday
3. Sunday

Selected schedule name: Monday to Friday

Number of buses scheduled between the first time available in the current slot and now: 4

Timetable:
Time   Line ID  Stop Point ID  Direction

Slot 11:
11:09   d7    490000038F  Inbound
11:21   d7    490000038F  Inbound
11:33   d7    490000038F  Inbound
11:45   d7    490000038F  Inbound
11:57   d7    490000038F  Inbound

Slot 12:
12:09   d7    490000038F  Inbound
12:21   d7    490000038F  Inbound
12:33   d7    490000038F  Inbound
12:45   d7    490000038F  Inbound
12:57   d7    490000038F  Inbound

Slot 13:
13:09   d7    490000038F  Inbound
13:21   d7    490000038F  Inbound
13:33   d7    490000038F  Inbound
13:45   d7    490000038F  Inbound
13:57   d7    490000038F  Inbound


Monday-Tuesday-Wednesday

In [21]:
import requests
from datetime import datetime
import pytz

# Function to fetch data from the API
def fetch_data(url):
    response = requests.get(url)
    return response.json()

# Function to extract schedule names
def extract_schedule_names(data, schedule_names_dict={}):
    if isinstance(data, dict):
        if data.get('$type') == "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities" and 'knownJourneys' in data:
            if 'name' in data:
                schedule_names_dict[data['name']] = data['knownJourneys']
        for key, value in data.items():
            extract_schedule_names(value, schedule_names_dict)
    elif isinstance(data, list):
        for item in data:
            extract_schedule_names(item, schedule_names_dict)
    return schedule_names_dict

# Function to categorize journeys into hourly slots
def categorize_into_slots(timetable):
    slots = [[] for _ in range(24)]
    for journey in timetable:
        hour = int(journey['hour'])  # Convert hour to integer
        if 0 <= hour < 24:  # Ensure hour is within the valid range
            slots[hour].append(journey)
    return slots

# Function to calculate the number of buses between the first time available in the slot and the current time
def calculate_buses(slot, current_hour, current_minute):
    bus_count = 0
    for journey in slot:
        journey_hour = int(journey['hour'])
        journey_minute = int(journey['minute'])
        if journey_hour == current_hour and journey_minute <= current_minute:
            bus_count += 1
    return bus_count

# Function to fetch the current day of the week
def get_day_of_week():
    bst = pytz.timezone('Europe/London')
    now = datetime.now(bst)
    return now.strftime('%A')  # %A gives full weekday name (e.g., 'Monday')

# Main function to get user input and print the schedule names and timetable
def main():
    line_id = input("Enter the line ID: ")
    stop_point_id = input("Enter the stop point ID: ")
    direction = input("Enter the direction (outbound/inbound): ")

    # Fetching data
    url = f'https://api.tfl.gov.uk/Line/{line_id}/Timetable/{stop_point_id}?direction={direction}'
    data = fetch_data(url)

    # Extracting schedule names
    schedule_names_dict = extract_schedule_names(data)

    # Fetch day of the week
    day_of_week = get_day_of_week()
    print(f"Today is {day_of_week}.")

    # Print extracted schedule names
    print("\nExtracted Schedule Names:")
    for i, schedule_name in enumerate(schedule_names_dict.keys(), start=1):
        print(f"{i}. {schedule_name}")

    # Determine the matching schedule name based on the current day of the week
    selected_schedule_name = None

    if day_of_week.lower() == 'monday' or day_of_week.lower() == 'tuesday' or day_of_week.lower() == 'wednesday':
        # Check for the preferred schedule names for Monday, Tuesday, and Wednesday
        preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Thursday', 'Monday to Friday']
        for preferred_name in preferred_schedule_names:
            if preferred_name in schedule_names_dict:
                selected_schedule_name = preferred_name
                break
    else:
        # For other days of the week, select based on exact match
        for schedule_name in schedule_names_dict.keys():
            if day_of_week.lower() in schedule_name.lower():
                selected_schedule_name = schedule_name
                break

    if selected_schedule_name:
        print(f"\nSelected schedule name: {selected_schedule_name}")

        # Categorizing the timetable into hourly slots
        timetable = schedule_names_dict[selected_schedule_name]
        slots = categorize_into_slots(timetable)

        # Get the current time in BST
        bst = pytz.timezone('Europe/London')
        now = datetime.now(bst)
        current_hour = now.hour
        current_minute = now.minute

        # Calculate the number of buses scheduled between the first time available in the current slot and the current time
        bus_count = calculate_buses(slots[current_hour], current_hour, current_minute)
        print(f"\nNumber of buses scheduled between the first time available in the current slot and now: {bus_count}")

        # Print the timetable slots corresponding to the current hour, the previous hour, and the next hour
        hours_to_print = [current_hour - 1, current_hour, current_hour + 1]
        print("\nTimetable:")
        print("Time   Line ID  Stop Point ID  Direction")
        for hour in hours_to_print:
            if 0 <= hour < 24:
                print(f"\nSlot {hour}:")
                for journey in slots[hour]:
                    journey_hour = str(journey['hour']).zfill(2)
                    journey_minute = str(journey['minute']).zfill(2)
                    time = f"{journey_hour}:{journey_minute}"
                    print(f"{time}   {line_id}    {stop_point_id}  {direction.capitalize()}")

    else:
        print("No matching schedule name found for today.")

# Run the main function
if __name__ == "__main__":
    main()


Enter the line ID: 179
Enter the stop point ID: 490005183E
Enter the direction (outbound/inbound): outbound
Today is Monday.

Extracted Schedule Names:
1. Mon-Fri Non-Schooldays
2. Mon-Fri Schooldays
3. Saturday
4. Sunday

Selected schedule name: Mon-Fri Schooldays

Number of buses scheduled between the first time available in the current slot and now: 4

Timetable:
Time   Line ID  Stop Point ID  Direction

Slot 11:
11:10   179    490005183E  Outbound
11:22   179    490005183E  Outbound
11:34   179    490005183E  Outbound
11:46   179    490005183E  Outbound
11:58   179    490005183E  Outbound

Slot 12:
12:10   179    490005183E  Outbound
12:22   179    490005183E  Outbound
12:34   179    490005183E  Outbound
12:46   179    490005183E  Outbound
12:58   179    490005183E  Outbound

Slot 13:
13:10   179    490005183E  Outbound
13:22   179    490005183E  Outbound
13:34   179    490005183E  Outbound
13:46   179    490005183E  Outbound
13:58   179    490005183E  Outbound


Trial for Wednesday

In [24]:
import requests
from datetime import datetime
import pytz

# Function to fetch data from the API
def fetch_data(url):
    response = requests.get(url)
    return response.json()

# Function to extract schedule names
def extract_schedule_names(data, schedule_names_dict={}):
    if isinstance(data, dict):
        if data.get('$type') == "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities" and 'knownJourneys' in data:
            if 'name' in data:
                schedule_names_dict[data['name']] = data['knownJourneys']
        for key, value in data.items():
            extract_schedule_names(value, schedule_names_dict)
    elif isinstance(data, list):
        for item in data:
            extract_schedule_names(item, schedule_names_dict)
    return schedule_names_dict

# Function to categorize journeys into hourly slots
def categorize_into_slots(timetable):
    slots = [[] for _ in range(24)]
    for journey in timetable:
        hour = int(journey['hour'])  # Convert hour to integer
        if 0 <= hour < 24:  # Ensure hour is within the valid range
            slots[hour].append(journey)
    return slots

# Function to calculate the number of buses between the first time available in the slot and the current time
def calculate_buses(slot, current_hour, current_minute):
    bus_count = 0
    for journey in slot:
        journey_hour = int(journey['hour'])
        journey_minute = int(journey['minute'])
        if journey_hour == current_hour and journey_minute <= current_minute:
            bus_count += 1
    return bus_count

# Function to fetch the current day of the week
def get_day_of_week():
    # Temporarily return 'Wednesday' instead of actual day of the week
    return 'Wednesday'

# Main function to get user input and print the schedule names and timetable
def main():
    line_id = input("Enter the line ID: ")
    stop_point_id = input("Enter the stop point ID: ")
    direction = input("Enter the direction (outbound/inbound): ")

    # Fetching data
    url = f'https://api.tfl.gov.uk/Line/{line_id}/Timetable/{stop_point_id}?direction={direction}'
    data = fetch_data(url)

    # Extracting schedule names
    schedule_names_dict = extract_schedule_names(data)

    # Fetch day of the week
    day_of_week = get_day_of_week()
    print(f"Today is {day_of_week}.")

    # Print extracted schedule names
    print("\nExtracted Schedule Names:")
    for i, schedule_name in enumerate(schedule_names_dict.keys(), start=1):
        print(f"{i}. {schedule_name}")

    # Determine the matching schedule name based on the current day of the week
    selected_schedule_name = None

    if day_of_week.lower() == 'monday' or day_of_week.lower() == 'tuesday' or day_of_week.lower() == 'wednesday':
        # Check for the preferred schedule names for Monday, Tuesday, and Wednesday
        preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Thursday', 'Monday to Friday']
        for preferred_name in preferred_schedule_names:
            if preferred_name in schedule_names_dict:
                selected_schedule_name = preferred_name
                break
    else:
        # For other days of the week, select based on exact match
        for schedule_name in schedule_names_dict.keys():
            if day_of_week.lower() in schedule_name.lower():
                selected_schedule_name = schedule_name
                break

    if selected_schedule_name:
        print(f"\nSelected schedule name: {selected_schedule_name}")

        # Categorizing the timetable into hourly slots
        timetable = schedule_names_dict[selected_schedule_name]
        slots = categorize_into_slots(timetable)

        # Get the current time in BST
        bst = pytz.timezone('Europe/London')
        now = datetime.now(bst)
        current_hour = now.hour
        current_minute = now.minute

        # Calculate the number of buses scheduled between the first time available in the current slot and the current time
        bus_count = calculate_buses(slots[current_hour], current_hour, current_minute)
        print(f"\nNumber of buses scheduled between the first time available in the current slot and now: {bus_count}")

        # Print the timetable slots corresponding to the current hour, the previous hour, and the next hour
        hours_to_print = [current_hour - 1, current_hour, current_hour + 1]
        print("\nTimetable:")
        print("Time   Line ID  Stop Point ID  Direction")
        for hour in hours_to_print:
            if 0 <= hour < 24:
                print(f"\nSlot {hour}:")
                for journey in slots[hour]:
                    journey_hour = str(journey['hour']).zfill(2)
                    journey_minute = str(journey['minute']).zfill(2)
                    time = f"{journey_hour}:{journey_minute}"
                    print(f"{time}   {line_id}    {stop_point_id}  {direction.capitalize()}")

    else:
        print("No matching schedule name found for today.")

# Run the main function
if __name__ == "__main__":
    main()


Enter the line ID: d7
Enter the stop point ID: 490000038F
Enter the direction (outbound/inbound): inbound
Today is Wednesday.

Extracted Schedule Names:
1. Monday to Friday
2. Saturday
3. Sunday

Selected schedule name: Monday to Friday

Number of buses scheduled between the first time available in the current slot and now: 4

Timetable:
Time   Line ID  Stop Point ID  Direction

Slot 11:
11:09   d7    490000038F  Inbound
11:21   d7    490000038F  Inbound
11:33   d7    490000038F  Inbound
11:45   d7    490000038F  Inbound
11:57   d7    490000038F  Inbound

Slot 12:
12:09   d7    490000038F  Inbound
12:21   d7    490000038F  Inbound
12:33   d7    490000038F  Inbound
12:45   d7    490000038F  Inbound
12:57   d7    490000038F  Inbound

Slot 13:
13:09   d7    490000038F  Inbound
13:21   d7    490000038F  Inbound
13:33   d7    490000038F  Inbound
13:45   d7    490000038F  Inbound
13:57   d7    490000038F  Inbound


Monday-Tuesday-Wednesday-Thursday

In [25]:
import requests
from datetime import datetime
import pytz

# Function to fetch data from the API
def fetch_data(url):
    response = requests.get(url)
    return response.json()

# Function to extract schedule names
def extract_schedule_names(data, schedule_names_dict={}):
    if isinstance(data, dict):
        if data.get('$type') == "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities" and 'knownJourneys' in data:
            if 'name' in data:
                schedule_names_dict[data['name']] = data['knownJourneys']
        for key, value in data.items():
            extract_schedule_names(value, schedule_names_dict)
    elif isinstance(data, list):
        for item in data:
            extract_schedule_names(item, schedule_names_dict)
    return schedule_names_dict

# Function to categorize journeys into hourly slots
def categorize_into_slots(timetable):
    slots = [[] for _ in range(24)]
    for journey in timetable:
        hour = int(journey['hour'])  # Convert hour to integer
        if 0 <= hour < 24:  # Ensure hour is within the valid range
            slots[hour].append(journey)
    return slots

# Function to calculate the number of buses between the first time available in the slot and the current time
def calculate_buses(slot, current_hour, current_minute):
    bus_count = 0
    for journey in slot:
        journey_hour = int(journey['hour'])
        journey_minute = int(journey['minute'])
        if journey_hour == current_hour and journey_minute <= current_minute:
            bus_count += 1
    return bus_count

# Function to fetch the current day of the week
def get_day_of_week():
    bst = pytz.timezone('Europe/London')
    now = datetime.now(bst)
    return now.strftime('%A')  # %A gives full weekday name (e.g., 'Monday')

# Main function to get user input and print the schedule names and timetable
def main():
    line_id = input("Enter the line ID: ")
    stop_point_id = input("Enter the stop point ID: ")
    direction = input("Enter the direction (outbound/inbound): ")

    # Fetching data
    url = f'https://api.tfl.gov.uk/Line/{line_id}/Timetable/{stop_point_id}?direction={direction}'
    data = fetch_data(url)

    # Extracting schedule names
    schedule_names_dict = extract_schedule_names(data)

    # Fetch day of the week
    day_of_week = get_day_of_week()
    print(f"Today is {day_of_week}.")

    # Print extracted schedule names
    print("\nExtracted Schedule Names:")
    for i, schedule_name in enumerate(schedule_names_dict.keys(), start=1):
        print(f"{i}. {schedule_name}")

    # Determine the matching schedule name based on the current day of the week
    selected_schedule_name = None

    if day_of_week.lower() in ['monday', 'tuesday', 'wednesday', 'thursday']:
        # Check for the preferred schedule names for Monday to Thursday
        preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Thursday', 'Monday to Friday']
        for preferred_name in preferred_schedule_names:
            if preferred_name in schedule_names_dict:
                selected_schedule_name = preferred_name
                break
    else:
        # For other days of the week, select based on exact match
        for schedule_name in schedule_names_dict.keys():
            if day_of_week.lower() in schedule_name.lower():
                selected_schedule_name = schedule_name
                break

    if selected_schedule_name:
        print(f"\nSelected schedule name: {selected_schedule_name}")

        # Categorizing the timetable into hourly slots
        timetable = schedule_names_dict[selected_schedule_name]
        slots = categorize_into_slots(timetable)

        # Get the current time in BST
        bst = pytz.timezone('Europe/London')
        now = datetime.now(bst)
        current_hour = now.hour
        current_minute = now.minute

        # Calculate the number of buses scheduled between the first time available in the current slot and the current time
        bus_count = calculate_buses(slots[current_hour], current_hour, current_minute)
        print(f"\nNumber of buses scheduled between the first time available in the current slot and now: {bus_count}")

        # Print the timetable slots corresponding to the current hour, the previous hour, and the next hour
        hours_to_print = [current_hour - 1, current_hour, current_hour + 1]
        print("\nTimetable:")
        print("Time   Line ID  Stop Point ID  Direction")
        for hour in hours_to_print:
            if 0 <= hour < 24:
                print(f"\nSlot {hour}:")
                for journey in slots[hour]:
                    journey_hour = str(journey['hour']).zfill(2)
                    journey_minute = str(journey['minute']).zfill(2)
                    time = f"{journey_hour}:{journey_minute}"
                    print(f"{time}   {line_id}    {stop_point_id}  {direction.capitalize()}")

    else:
        print("No matching schedule name found for today.")

# Run the main function
if __name__ == "__main__":
    main()


Enter the line ID: d7
Enter the stop point ID: 490000038F
Enter the direction (outbound/inbound): inbound
Today is Monday.

Extracted Schedule Names:
1. Monday to Friday
2. Saturday
3. Sunday

Selected schedule name: Monday to Friday

Number of buses scheduled between the first time available in the current slot and now: 4

Timetable:
Time   Line ID  Stop Point ID  Direction

Slot 11:
11:09   d7    490000038F  Inbound
11:21   d7    490000038F  Inbound
11:33   d7    490000038F  Inbound
11:45   d7    490000038F  Inbound
11:57   d7    490000038F  Inbound

Slot 12:
12:09   d7    490000038F  Inbound
12:21   d7    490000038F  Inbound
12:33   d7    490000038F  Inbound
12:45   d7    490000038F  Inbound
12:57   d7    490000038F  Inbound

Slot 13:
13:09   d7    490000038F  Inbound
13:21   d7    490000038F  Inbound
13:33   d7    490000038F  Inbound
13:45   d7    490000038F  Inbound
13:57   d7    490000038F  Inbound


Trial for Thursday

In [28]:
import requests
from datetime import datetime
import pytz

# Function to fetch data from the API
def fetch_data(url):
    response = requests.get(url)
    return response.json()

# Function to extract schedule names
def extract_schedule_names(data, schedule_names_dict={}):
    if isinstance(data, dict):
        if data.get('$type') == "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities" and 'knownJourneys' in data:
            if 'name' in data:
                schedule_names_dict[data['name']] = data['knownJourneys']
        for key, value in data.items():
            extract_schedule_names(value, schedule_names_dict)
    elif isinstance(data, list):
        for item in data:
            extract_schedule_names(item, schedule_names_dict)
    return schedule_names_dict

# Function to categorize journeys into hourly slots
def categorize_into_slots(timetable):
    slots = [[] for _ in range(24)]
    for journey in timetable:
        hour = int(journey['hour'])  # Convert hour to integer
        if 0 <= hour < 24:  # Ensure hour is within the valid range
            slots[hour].append(journey)
    return slots

# Function to calculate the number of buses between the first time available in the slot and the current time
def calculate_buses(slot, current_hour, current_minute):
    bus_count = 0
    for journey in slot:
        journey_hour = int(journey['hour'])
        journey_minute = int(journey['minute'])
        if journey_hour == current_hour and journey_minute <= current_minute:
            bus_count += 1
    return bus_count

# Function to fetch the current day of the week
def get_day_of_week():
    # Temporarily return 'Thursday' to simulate testing for Thursday
    return 'Thursday'

# Main function to get user input and print the schedule names and timetable
def main():
    line_id = input("Enter the line ID: ")
    stop_point_id = input("Enter the stop point ID: ")
    direction = input("Enter the direction (outbound/inbound): ")

    # Fetching data
    url = f'https://api.tfl.gov.uk/Line/{line_id}/Timetable/{stop_point_id}?direction={direction}'
    data = fetch_data(url)

    # Extracting schedule names
    schedule_names_dict = extract_schedule_names(data)

    # Fetch day of the week
    day_of_week = get_day_of_week()
    print(f"Today is {day_of_week}.")

    # Print extracted schedule names
    print("\nExtracted Schedule Names:")
    for i, schedule_name in enumerate(schedule_names_dict.keys(), start=1):
        print(f"{i}. {schedule_name}")

    # Determine the matching schedule name based on the current day of the week
    selected_schedule_name = None

    if day_of_week.lower() in ['monday', 'tuesday', 'wednesday', 'thursday']:
        # Check for the preferred schedule names for Monday to Thursday
        preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Thursday', 'Monday to Friday']
        for preferred_name in preferred_schedule_names:
            if preferred_name in schedule_names_dict:
                selected_schedule_name = preferred_name
                break
    else:
        # For other days of the week, select based on exact match
        for schedule_name in schedule_names_dict.keys():
            if day_of_week.lower() in schedule_name.lower():
                selected_schedule_name = schedule_name
                break

    if selected_schedule_name:
        print(f"\nSelected schedule name: {selected_schedule_name}")

        # Categorizing the timetable into hourly slots
        timetable = schedule_names_dict[selected_schedule_name]
        slots = categorize_into_slots(timetable)

        # Get the current time in BST
        bst = pytz.timezone('Europe/London')
        now = datetime.now(bst)
        current_hour = now.hour
        current_minute = now.minute

        # Calculate the number of buses scheduled between the first time available in the current slot and the current time
        bus_count = calculate_buses(slots[current_hour], current_hour, current_minute)
        print(f"\nNumber of buses scheduled between the first time available in the current slot and now: {bus_count}")

        # Print the timetable slots corresponding to the current hour, the previous hour, and the next hour
        hours_to_print = [current_hour - 1, current_hour, current_hour + 1]
        print("\nTimetable:")
        print("Time   Line ID  Stop Point ID  Direction")
        for hour in hours_to_print:
            if 0 <= hour < 24:
                print(f"\nSlot {hour}:")
                for journey in slots[hour]:
                    journey_hour = str(journey['hour']).zfill(2)
                    journey_minute = str(journey['minute']).zfill(2)
                    time = f"{journey_hour}:{journey_minute}"
                    print(f"{time}   {line_id}    {stop_point_id}  {direction.capitalize()}")

    else:
        print("No matching schedule name found for today.")

# Run the main function
if __name__ == "__main__":
    main()


Enter the line ID: d7
Enter the stop point ID: 490000038F
Enter the direction (outbound/inbound): inbound
Today is Thursday.

Extracted Schedule Names:
1. Monday to Friday
2. Saturday
3. Sunday

Selected schedule name: Monday to Friday

Number of buses scheduled between the first time available in the current slot and now: 5

Timetable:
Time   Line ID  Stop Point ID  Direction

Slot 11:
11:09   d7    490000038F  Inbound
11:21   d7    490000038F  Inbound
11:33   d7    490000038F  Inbound
11:45   d7    490000038F  Inbound
11:57   d7    490000038F  Inbound

Slot 12:
12:09   d7    490000038F  Inbound
12:21   d7    490000038F  Inbound
12:33   d7    490000038F  Inbound
12:45   d7    490000038F  Inbound
12:57   d7    490000038F  Inbound

Slot 13:
13:09   d7    490000038F  Inbound
13:21   d7    490000038F  Inbound
13:33   d7    490000038F  Inbound
13:45   d7    490000038F  Inbound
13:57   d7    490000038F  Inbound


Monday-Tuesday-Wednesday-Thursday-Friday

In [29]:
import requests
from datetime import datetime
import pytz

# Function to fetch data from the API
def fetch_data(url):
    response = requests.get(url)
    return response.json()

# Function to extract schedule names
def extract_schedule_names(data, schedule_names_dict={}):
    if isinstance(data, dict):
        if data.get('$type') == "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities" and 'knownJourneys' in data:
            if 'name' in data:
                schedule_names_dict[data['name']] = data['knownJourneys']
        for key, value in data.items():
            extract_schedule_names(value, schedule_names_dict)
    elif isinstance(data, list):
        for item in data:
            extract_schedule_names(item, schedule_names_dict)
    return schedule_names_dict

# Function to categorize journeys into hourly slots
def categorize_into_slots(timetable):
    slots = [[] for _ in range(24)]
    for journey in timetable:
        hour = int(journey['hour'])  # Convert hour to integer
        if 0 <= hour < 24:  # Ensure hour is within the valid range
            slots[hour].append(journey)
    return slots

# Function to calculate the number of buses between the first time available in the slot and the current time
def calculate_buses(slot, current_hour, current_minute):
    bus_count = 0
    for journey in slot:
        journey_hour = int(journey['hour'])
        journey_minute = int(journey['minute'])
        if journey_hour == current_hour and journey_minute <= current_minute:
            bus_count += 1
    return bus_count

# Function to fetch the current day of the week
def get_day_of_week():
    bst = pytz.timezone('Europe/London')
    now = datetime.now(bst)
    return now.strftime('%A')  # %A gives full weekday name (e.g., 'Monday')

# Main function to get user input and print the schedule names and timetable
def main():
    line_id = input("Enter the line ID: ")
    stop_point_id = input("Enter the stop point ID: ")
    direction = input("Enter the direction (outbound/inbound): ")

    # Fetching data
    url = f'https://api.tfl.gov.uk/Line/{line_id}/Timetable/{stop_point_id}?direction={direction}'
    data = fetch_data(url)

    # Extracting schedule names
    schedule_names_dict = extract_schedule_names(data)

    # Fetch day of the week
    day_of_week = get_day_of_week()
    print(f"Today is {day_of_week}.")

    # Print extracted schedule names
    print("\nExtracted Schedule Names:")
    for i, schedule_name in enumerate(schedule_names_dict.keys(), start=1):
        print(f"{i}. {schedule_name}")

    # Determine the matching schedule name based on the current day of the week
    selected_schedule_name = None

    if day_of_week.lower() in ['monday', 'tuesday', 'wednesday', 'thursday']:
        # Check for the preferred schedule names for Monday to Thursday
        preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Thursday', 'Monday to Friday']
        for preferred_name in preferred_schedule_names:
            if preferred_name in schedule_names_dict:
                selected_schedule_name = preferred_name
                break
    elif day_of_week.lower() == 'friday':
        # Check for the preferred schedule names for Friday
        preferred_schedule_names = ['Mon-Fri Schooldays', 'Friday', 'Monday to Friday']
        for preferred_name in preferred_schedule_names:
            if preferred_name in schedule_names_dict:
                selected_schedule_name = preferred_name
                break
    else:
        # For other days of the week, select based on exact match
        for schedule_name in schedule_names_dict.keys():
            if day_of_week.lower() in schedule_name.lower():
                selected_schedule_name = schedule_name
                break

    if selected_schedule_name:
        print(f"\nSelected schedule name: {selected_schedule_name}")

        # Categorizing the timetable into hourly slots
        timetable = schedule_names_dict[selected_schedule_name]
        slots = categorize_into_slots(timetable)

        # Get the current time in BST
        bst = pytz.timezone('Europe/London')
        now = datetime.now(bst)
        current_hour = now.hour
        current_minute = now.minute

        # Calculate the number of buses scheduled between the first time available in the current slot and the current time
        bus_count = calculate_buses(slots[current_hour], current_hour, current_minute)
        print(f"\nNumber of buses scheduled between the first time available in the current slot and now: {bus_count}")

        # Print the timetable slots corresponding to the current hour, the previous hour, and the next hour
        hours_to_print = [current_hour - 1, current_hour, current_hour + 1]
        print("\nTimetable:")
        print("Time   Line ID  Stop Point ID  Direction")
        for hour in hours_to_print:
            if 0 <= hour < 24:
                print(f"\nSlot {hour}:")
                for journey in slots[hour]:
                    journey_hour = str(journey['hour']).zfill(2)
                    journey_minute = str(journey['minute']).zfill(2)
                    time = f"{journey_hour}:{journey_minute}"
                    print(f"{time}   {line_id}    {stop_point_id}  {direction.capitalize()}")

    else:
        print("No matching schedule name found for today.")

# Run the main function
if __name__ == "__main__":
    main()


Enter the line ID: d7
Enter the stop point ID: 490000038F
Enter the direction (outbound/inbound): inbound
Today is Monday.

Extracted Schedule Names:
1. Monday to Friday
2. Saturday
3. Sunday

Selected schedule name: Monday to Friday

Number of buses scheduled between the first time available in the current slot and now: 0

Timetable:
Time   Line ID  Stop Point ID  Direction

Slot 12:
12:09   d7    490000038F  Inbound
12:21   d7    490000038F  Inbound
12:33   d7    490000038F  Inbound
12:45   d7    490000038F  Inbound
12:57   d7    490000038F  Inbound

Slot 13:
13:09   d7    490000038F  Inbound
13:21   d7    490000038F  Inbound
13:33   d7    490000038F  Inbound
13:45   d7    490000038F  Inbound
13:57   d7    490000038F  Inbound

Slot 14:
14:09   d7    490000038F  Inbound
14:21   d7    490000038F  Inbound
14:33   d7    490000038F  Inbound
14:44   d7    490000038F  Inbound
14:54   d7    490000038F  Inbound


Trial for Friday

In [32]:
import requests
from datetime import datetime
import pytz

# Function to fetch data from the API
def fetch_data(url):
    response = requests.get(url)
    return response.json()

# Function to extract schedule names
def extract_schedule_names(data, schedule_names_dict={}):
    if isinstance(data, dict):
        if data.get('$type') == "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities" and 'knownJourneys' in data:
            if 'name' in data:
                schedule_names_dict[data['name']] = data['knownJourneys']
        for key, value in data.items():
            extract_schedule_names(value, schedule_names_dict)
    elif isinstance(data, list):
        for item in data:
            extract_schedule_names(item, schedule_names_dict)
    return schedule_names_dict

# Function to categorize journeys into hourly slots
def categorize_into_slots(timetable):
    slots = [[] for _ in range(24)]
    for journey in timetable:
        hour = int(journey['hour'])  # Convert hour to integer
        if 0 <= hour < 24:  # Ensure hour is within the valid range
            slots[hour].append(journey)
    return slots

# Function to calculate the number of buses between the first time available in the slot and the current time
def calculate_buses(slot, current_hour, current_minute):
    bus_count = 0
    for journey in slot:
        journey_hour = int(journey['hour'])
        journey_minute = int(journey['minute'])
        if journey_hour == current_hour and journey_minute <= current_minute:
            bus_count += 1
    return bus_count

# Function to fetch the current day of the week
def get_day_of_week():
    return 'Friday'  # Mocking Friday for testing

# Main function to get user input and print the schedule names and timetable
def main():
    line_id = input("Enter the line ID: ")
    stop_point_id = input("Enter the stop point ID: ")
    direction = input("Enter the direction (outbound/inbound): ")

    # Fetching data
    url = f'https://api.tfl.gov.uk/Line/{line_id}/Timetable/{stop_point_id}?direction={direction}'
    data = fetch_data(url)

    # Extracting schedule names
    schedule_names_dict = extract_schedule_names(data)

    # Fetch day of the week (mocked for testing)
    day_of_week = get_day_of_week()
    print(f"Today is {day_of_week}.")

    # Print extracted schedule names
    print("\nExtracted Schedule Names:")
    for i, schedule_name in enumerate(schedule_names_dict.keys(), start=1):
        print(f"{i}. {schedule_name}")

    # Determine the matching schedule name based on the current day of the week
    selected_schedule_name = None

    if day_of_week.lower() in ['monday', 'tuesday', 'wednesday', 'thursday', 'friday']:
        # Check for the preferred schedule names for Monday to Friday
        preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Thursday', 'Monday to Friday', 'Friday']
        for preferred_name in preferred_schedule_names:
            if preferred_name in schedule_names_dict:
                selected_schedule_name = preferred_name
                break

    if selected_schedule_name:
        print(f"\nSelected schedule name: {selected_schedule_name}")

        # Categorizing the timetable into hourly slots
        timetable = schedule_names_dict[selected_schedule_name]
        slots = categorize_into_slots(timetable)

        # Get the current time in BST
        bst = pytz.timezone('Europe/London')
        now = datetime.now(bst)
        current_hour = now.hour
        current_minute = now.minute

        # Calculate the number of buses scheduled between the first time available in the current slot and the current time
        bus_count = calculate_buses(slots[current_hour], current_hour, current_minute)
        print(f"\nNumber of buses scheduled between the first time available in the current slot and now: {bus_count}")

        # Print the timetable slots corresponding to the current hour, the previous hour, and the next hour
        hours_to_print = [current_hour - 1, current_hour, current_hour + 1]
        print("\nTimetable:")
        print("Time   Line ID  Stop Point ID  Direction")
        for hour in hours_to_print:
            if 0 <= hour < 24:
                print(f"\nSlot {hour}:")
                for journey in slots[hour]:
                    journey_hour = str(journey['hour']).zfill(2)
                    journey_minute = str(journey['minute']).zfill(2)
                    time = f"{journey_hour}:{journey_minute}"
                    print(f"{time}   {line_id}    {stop_point_id}  {direction.capitalize()}")

    else:
        print("No matching schedule name found for today.")

# Run the main function
if __name__ == "__main__":
    main()


Enter the line ID: d7
Enter the stop point ID: 490000038F
Enter the direction (outbound/inbound): inbound
Today is Friday.

Extracted Schedule Names:
1. Monday to Friday
2. Saturday
3. Sunday

Selected schedule name: Monday to Friday

Number of buses scheduled between the first time available in the current slot and now: 1

Timetable:
Time   Line ID  Stop Point ID  Direction

Slot 12:
12:09   d7    490000038F  Inbound
12:21   d7    490000038F  Inbound
12:33   d7    490000038F  Inbound
12:45   d7    490000038F  Inbound
12:57   d7    490000038F  Inbound

Slot 13:
13:09   d7    490000038F  Inbound
13:21   d7    490000038F  Inbound
13:33   d7    490000038F  Inbound
13:45   d7    490000038F  Inbound
13:57   d7    490000038F  Inbound

Slot 14:
14:09   d7    490000038F  Inbound
14:21   d7    490000038F  Inbound
14:33   d7    490000038F  Inbound
14:44   d7    490000038F  Inbound
14:54   d7    490000038F  Inbound


Monday-Tuesday-Wednesday-Thursday-Friday-Saturday

In [33]:
import requests
from datetime import datetime
import pytz

# Function to fetch data from the API
def fetch_data(url):
    response = requests.get(url)
    return response.json()

# Function to extract schedule names
def extract_schedule_names(data, schedule_names_dict={}):
    if isinstance(data, dict):
        if data.get('$type') == "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities" and 'knownJourneys' in data:
            if 'name' in data:
                schedule_names_dict[data['name']] = data['knownJourneys']
        for key, value in data.items():
            extract_schedule_names(value, schedule_names_dict)
    elif isinstance(data, list):
        for item in data:
            extract_schedule_names(item, schedule_names_dict)
    return schedule_names_dict

# Function to categorize journeys into hourly slots
def categorize_into_slots(timetable):
    slots = [[] for _ in range(24)]
    for journey in timetable:
        hour = int(journey['hour'])  # Convert hour to integer
        if 0 <= hour < 24:  # Ensure hour is within the valid range
            slots[hour].append(journey)
    return slots

# Function to calculate the number of buses between the first time available in the slot and the current time
def calculate_buses(slot, current_hour, current_minute):
    bus_count = 0
    for journey in slot:
        journey_hour = int(journey['hour'])
        journey_minute = int(journey['minute'])
        if journey_hour == current_hour and journey_minute <= current_minute:
            bus_count += 1
    return bus_count

# Function to fetch the current day of the week
def get_day_of_week():
    bst = pytz.timezone('Europe/London')
    now = datetime.now(bst)
    return now.strftime('%A')  # %A gives full weekday name (e.g., 'Monday')

# Main function to get user input and print the schedule names and timetable
def main():
    line_id = input("Enter the line ID: ")
    stop_point_id = input("Enter the stop point ID: ")
    direction = input("Enter the direction (outbound/inbound): ")

    # Fetching data
    url = f'https://api.tfl.gov.uk/Line/{line_id}/Timetable/{stop_point_id}?direction={direction}'
    data = fetch_data(url)

    # Extracting schedule names
    schedule_names_dict = extract_schedule_names(data)

    # Fetch day of the week
    day_of_week = get_day_of_week()
    print(f"Today is {day_of_week}.")

    # Print extracted schedule names
    print("\nExtracted Schedule Names:")
    for i, schedule_name in enumerate(schedule_names_dict.keys(), start=1):
        print(f"{i}. {schedule_name}")

    # Determine the matching schedule name based on the current day of the week
    selected_schedule_name = None

    if day_of_week.lower() == 'monday' or day_of_week.lower() == 'tuesday' or day_of_week.lower() == 'wednesday' or day_of_week.lower() == 'thursday':
        # Check for the preferred schedule names for Monday to Thursday
        preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Thursday', 'Monday to Friday']
        for preferred_name in preferred_schedule_names:
            if preferred_name in schedule_names_dict:
                selected_schedule_name = preferred_name
                break
    elif day_of_week.lower() == 'friday':
        # Check for the preferred schedule names for Friday
        preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Friday', 'Friday']
        for preferred_name in preferred_schedule_names:
            if preferred_name in schedule_names_dict:
                selected_schedule_name = preferred_name
                break
    elif day_of_week.lower() == 'saturday':
        # Check for the preferred schedule names for Saturday
        preferred_schedule_names = ['Saturday']
        for preferred_name in preferred_schedule_names:
            if preferred_name in schedule_names_dict:
                selected_schedule_name = preferred_name
                break
    else:
        # For other days of the week, select based on exact match
        for schedule_name in schedule_names_dict.keys():
            if day_of_week.lower() in schedule_name.lower():
                selected_schedule_name = schedule_name
                break

    if selected_schedule_name:
        print(f"\nSelected schedule name: {selected_schedule_name}")

        # Categorizing the timetable into hourly slots
        timetable = schedule_names_dict[selected_schedule_name]
        slots = categorize_into_slots(timetable)

        # Get the current time in BST
        bst = pytz.timezone('Europe/London')
        now = datetime.now(bst)
        current_hour = now.hour
        current_minute = now.minute

        # Calculate the number of buses scheduled between the first time available in the current slot and the current time
        bus_count = calculate_buses(slots[current_hour], current_hour, current_minute)
        print(f"\nNumber of buses scheduled between the first time available in the current slot and now: {bus_count}")

        # Print the timetable slots corresponding to the current hour, the previous hour, and the next hour
        hours_to_print = [current_hour - 1, current_hour, current_hour + 1]
        print("\nTimetable:")
        print("Time   Line ID  Stop Point ID  Direction")
        for hour in hours_to_print:
            if 0 <= hour < 24:
                print(f"\nSlot {hour}:")
                for journey in slots[hour]:
                    journey_hour = str(journey['hour']).zfill(2)
                    journey_minute = str(journey['minute']).zfill(2)
                    time = f"{journey_hour}:{journey_minute}"
                    print(f"{time}   {line_id}    {stop_point_id}  {direction.capitalize()}")

    else:
        print("No matching schedule name found for today.")

# Run the main function
if __name__ == "__main__":
    main()


Enter the line ID: 22
Enter the stop point ID: 490012375E
Enter the direction (outbound/inbound): outbound
Today is Monday.

Extracted Schedule Names:
1. Friday
2. Monday to Thursday
3. Saturday
4. Sunday

Selected schedule name: Monday to Thursday

Number of buses scheduled between the first time available in the current slot and now: 1

Timetable:
Time   Line ID  Stop Point ID  Direction

Slot 12:
12:09   22    490012375E  Outbound
12:20   22    490012375E  Outbound
12:31   22    490012375E  Outbound
12:42   22    490012375E  Outbound
12:53   22    490012375E  Outbound

Slot 13:
13:05   22    490012375E  Outbound
13:16   22    490012375E  Outbound
13:28   22    490012375E  Outbound
13:39   22    490012375E  Outbound
13:51   22    490012375E  Outbound

Slot 14:
14:02   22    490012375E  Outbound
14:14   22    490012375E  Outbound
14:25   22    490012375E  Outbound
14:36   22    490012375E  Outbound
14:47   22    490012375E  Outbound
14:57   22    490012375E  Outbound


Trial for Saturday

In [36]:
import requests
from datetime import datetime
import pytz

# Function to fetch data from the API
def fetch_data(url):
    response = requests.get(url)
    return response.json()

# Function to extract schedule names
def extract_schedule_names(data, schedule_names_dict={}):
    if isinstance(data, dict):
        if data.get('$type') == "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities" and 'knownJourneys' in data:
            if 'name' in data:
                schedule_names_dict[data['name']] = data['knownJourneys']
        for key, value in data.items():
            extract_schedule_names(value, schedule_names_dict)
    elif isinstance(data, list):
        for item in data:
            extract_schedule_names(item, schedule_names_dict)
    return schedule_names_dict

# Function to categorize journeys into hourly slots
def categorize_into_slots(timetable):
    slots = [[] for _ in range(24)]
    for journey in timetable:
        hour = int(journey['hour'])  # Convert hour to integer
        if 0 <= hour < 24:  # Ensure hour is within the valid range
            slots[hour].append(journey)
    return slots

# Function to calculate the number of buses between the first time available in the slot and the current time
def calculate_buses(slot, current_hour, current_minute):
    bus_count = 0
    for journey in slot:
        journey_hour = int(journey['hour'])
        journey_minute = int(journey['minute'])
        if journey_hour == current_hour and journey_minute <= current_minute:
            bus_count += 1
    return bus_count

# Function to fetch the current day of the week
def get_day_of_week():
    return 'Saturday'  # Mocking Saturday for testing

# Main function to get user input and print the schedule names and timetable
def main():
    line_id = input("Enter the line ID: ")
    stop_point_id = input("Enter the stop point ID: ")
    direction = input("Enter the direction (outbound/inbound): ")

    # Fetching data
    url = f'https://api.tfl.gov.uk/Line/{line_id}/Timetable/{stop_point_id}?direction={direction}'
    data = fetch_data(url)

    # Extracting schedule names
    schedule_names_dict = extract_schedule_names(data)

    # Fetch day of the week (mocked for testing)
    day_of_week = get_day_of_week()
    print(f"Today is {day_of_week}.")

    # Print extracted schedule names
    print("\nExtracted Schedule Names:")
    for i, schedule_name in enumerate(schedule_names_dict.keys(), start=1):
        print(f"{i}. {schedule_name}")

    # Determine the matching schedule name based on the current day of the week
    selected_schedule_name = None

    if day_of_week.lower() == 'monday' or day_of_week.lower() == 'tuesday' or day_of_week.lower() == 'wednesday' or day_of_week.lower() == 'thursday':
        # Check for the preferred schedule names for Monday to Thursday
        preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Thursday', 'Monday to Friday', 'Friday']
        for preferred_name in preferred_schedule_names:
            if preferred_name in schedule_names_dict:
                selected_schedule_name = preferred_name
                break
    elif day_of_week.lower() == 'friday':
        # Check for the preferred schedule names for Friday
        preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Thursday', 'Monday to Friday', 'Friday']
        for preferred_name in preferred_schedule_names:
            if preferred_name in schedule_names_dict:
                selected_schedule_name = preferred_name
                break
    elif day_of_week.lower() == 'saturday':
        # Check for the preferred schedule names for Saturday
        preferred_schedule_names = ['Saturday']
        for preferred_name in preferred_schedule_names:
            if preferred_name in schedule_names_dict:
                selected_schedule_name = preferred_name
                break
    else:
        # For other days of the week, select based on exact match
        for schedule_name in schedule_names_dict.keys():
            if day_of_week.lower() in schedule_name.lower():
                selected_schedule_name = schedule_name
                break

    if selected_schedule_name:
        print(f"\nSelected schedule name: {selected_schedule_name}")

        # Categorizing the timetable into hourly slots
        timetable = schedule_names_dict[selected_schedule_name]
        slots = categorize_into_slots(timetable)

        # Get the current time in BST
        bst = pytz.timezone('Europe/London')
        now = datetime.now(bst)
        current_hour = now.hour
        current_minute = now.minute

        # Calculate the number of buses scheduled between the first time available in the current slot and the current time
        bus_count = calculate_buses(slots[current_hour], current_hour, current_minute)
        print(f"\nNumber of buses scheduled between the first time available in the current slot and now: {bus_count}")

        # Print the timetable slots corresponding to the current hour, the previous hour, and the next hour
        hours_to_print = [current_hour - 1, current_hour, current_hour + 1]
        print("\nTimetable:")
        print("Time   Line ID  Stop Point ID  Direction")
        for hour in hours_to_print:
            if 0 <= hour < 24:
                print(f"\nSlot {hour}:")
                for journey in slots[hour]:
                    journey_hour = str(journey['hour']).zfill(2)
                    journey_minute = str(journey['minute']).zfill(2)
                    time = f"{journey_hour}:{journey_minute}"
                    print(f"{time}   {line_id}    {stop_point_id}  {direction.capitalize()}")

    else:
        print("No matching schedule name found for today.")

# Run the main function
if __name__ == "__main__":
    main()


Enter the line ID: d7
Enter the stop point ID: 490000038F
Enter the direction (outbound/inbound): inbound
Today is Saturday.

Extracted Schedule Names:
1. Monday to Friday
2. Saturday
3. Sunday

Selected schedule name: Saturday

Number of buses scheduled between the first time available in the current slot and now: 2

Timetable:
Time   Line ID  Stop Point ID  Direction

Slot 12:
12:01   d7    490000038F  Inbound
12:14   d7    490000038F  Inbound
12:26   d7    490000038F  Inbound
12:38   d7    490000038F  Inbound
12:50   d7    490000038F  Inbound

Slot 13:
13:02   d7    490000038F  Inbound
13:14   d7    490000038F  Inbound
13:26   d7    490000038F  Inbound
13:38   d7    490000038F  Inbound
13:50   d7    490000038F  Inbound

Slot 14:
14:02   d7    490000038F  Inbound
14:14   d7    490000038F  Inbound
14:26   d7    490000038F  Inbound
14:38   d7    490000038F  Inbound
14:50   d7    490000038F  Inbound


Monday-Tuesday-Wednesday-Thursday-Friday-Saturday-Sunday

File share 16

In [ ]:
import requests
from datetime import datetime
import pytz

# Function to fetch data from the API
def fetch_data(url):
    response = requests.get(url)
    return response.json()

# Function to extract schedule names
def extract_schedule_names(data, schedule_names_dict={}):
    if isinstance(data, dict):
        if data.get('$type') == "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities" and 'knownJourneys' in data:
            if 'name' in data:
                schedule_names_dict[data['name']] = data['knownJourneys']
        for key, value in data.items():
            extract_schedule_names(value, schedule_names_dict)
    elif isinstance(data, list):
        for item in data:
            extract_schedule_names(item, schedule_names_dict)
    return schedule_names_dict

# Function to categorize journeys into hourly slots
def categorize_into_slots(timetable):
    slots = [[] for _ in range(24)]
    for journey in timetable:
        hour = int(journey['hour'])  # Convert hour to integer
        if 0 <= hour < 24:  # Ensure hour is within the valid range
            slots[hour].append(journey)
    return slots

# Function to calculate the number of buses between the first time available in the slot and the current time
def calculate_buses(slot, current_hour, current_minute):
    bus_count = 0
    for journey in slot:
        journey_hour = int(journey['hour'])
        journey_minute = int(journey['minute'])
        if journey_hour == current_hour and journey_minute <= current_minute:
            bus_count += 1
    return bus_count

# Function to fetch the current day of the week
def get_day_of_week():
    bst = pytz.timezone('Europe/London')
    now = datetime.now(bst)
    return now.strftime('%A')  # %A gives full weekday name (e.g., 'Monday')

# Main function to get user input and print the schedule names and timetable
def main():
    line_id = input("Enter the line ID: ")
    stop_point_id = input("Enter the stop point ID: ")
    direction = input("Enter the direction (outbound/inbound): ")

    # Fetching data
    url = f'https://api.tfl.gov.uk/Line/{line_id}/Timetable/{stop_point_id}?direction={direction}'
    data = fetch_data(url)

    # Extracting schedule names
    schedule_names_dict = extract_schedule_names(data)

    # Fetch day of the week
    day_of_week = get_day_of_week()
    print(f"Today is {day_of_week}.")

    # Print extracted schedule names
    print("\nExtracted Schedule Names:")
    for i, schedule_name in enumerate(schedule_names_dict.keys(), start=1):
        print(f"{i}. {schedule_name}")

    # Determine the matching schedule name based on the current day of the week
    selected_schedule_name = None

    if day_of_week.lower() == 'monday' or day_of_week.lower() == 'tuesday' or day_of_week.lower() == 'wednesday' or day_of_week.lower() == 'thursday':
        # Check for the preferred schedule names for Monday to Thursday
        preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Thursday', 'Monday to Friday']
        for preferred_name in preferred_schedule_names:
            if preferred_name in schedule_names_dict:
                selected_schedule_name = preferred_name
                break
    elif day_of_week.lower() == 'friday':
        # Check for the preferred schedule names for Friday
        preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Friday', 'Friday']
        for preferred_name in preferred_schedule_names:
            if preferred_name in schedule_names_dict:
                selected_schedule_name = preferred_name
                break
    elif day_of_week.lower() == 'saturday':
        # Check for the preferred schedule names for Saturday
        preferred_schedule_names = ['Saturday']
        for preferred_name in preferred_schedule_names:
            if preferred_name in schedule_names_dict:
                selected_schedule_name = preferred_name
                break
    elif day_of_week.lower() == 'sunday':
        # Check for the preferred schedule names for Sunday
        preferred_schedule_names = ['Sunday']
        for preferred_name in preferred_schedule_names:
            if preferred_name in schedule_names_dict:
                selected_schedule_name = preferred_name
                break
    else:
        # For other days of the week, select based on exact match
        for schedule_name in schedule_names_dict.keys():
            if day_of_week.lower() in schedule_name.lower():
                selected_schedule_name = schedule_name
                break

    if selected_schedule_name:
        print(f"\nSelected schedule name: {selected_schedule_name}")

        # Categorizing the timetable into hourly slots
        timetable = schedule_names_dict[selected_schedule_name]
        slots = categorize_into_slots(timetable)

        # Get the current time in BST
        bst = pytz.timezone('Europe/London')
        now = datetime.now(bst)
        current_hour = now.hour
        current_minute = now.minute

        # Calculate the number of buses scheduled between the first time available in the current slot and the current time
        bus_count = calculate_buses(slots[current_hour], current_hour, current_minute)
        print(f"\nNumber of buses scheduled between the first time available in the current slot and now: {bus_count}")

        # Print the timetable slots corresponding to the current hour, the previous hour, and the next hour
        hours_to_print = [current_hour - 1, current_hour, current_hour + 1]
        print("\nTimetable:")
        print("Time   Line ID  Stop Point ID  Direction")
        for hour in hours_to_print:
            if 0 <= hour < 24:
                print(f"\nSlot {hour}:")
                for journey in slots[hour]:
                    journey_hour = str(journey['hour']).zfill(2)
                    journey_minute = str(journey['minute']).zfill(2)
                    time = f"{journey_hour}:{journey_minute}"
                    print(f"{time}   {line_id}    {stop_point_id}  {direction.capitalize()}")

    else:
        print("No matching schedule name found for today.")

# Run the main function
if __name__ == "__main__":
    main()


Trial for Sunday

In [61]:
import requests
from datetime import datetime
import pytz

# Function to fetch data from the API
def fetch_data(url):
    response = requests.get(url)
    return response.json()

# Function to extract schedule names
def extract_schedule_names(data, schedule_names_dict={}):
    if isinstance(data, dict):
        if data.get('$type') == "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities" and 'knownJourneys' in data:
            if 'name' in data:
                schedule_names_dict[data['name']] = data['knownJourneys']
        for key, value in data.items():
            extract_schedule_names(value, schedule_names_dict)
    elif isinstance(data, list):
        for item in data:
            extract_schedule_names(item, schedule_names_dict)
    return schedule_names_dict

# Function to categorize journeys into hourly slots
def categorize_into_slots(timetable):
    slots = [[] for _ in range(24)]
    for journey in timetable:
        hour = int(journey['hour'])  # Convert hour to integer
        if 0 <= hour < 24:  # Ensure hour is within the valid range
            slots[hour].append(journey)
    return slots

# Function to calculate the number of buses between the first time available in the slot and the current time
def calculate_buses(slot, current_hour, current_minute):
    bus_count = 0
    for journey in slot:
        journey_hour = int(journey['hour'])
        journey_minute = int(journey['minute'])
        if journey_hour == current_hour and journey_minute <= current_minute:
            bus_count += 1
    return bus_count

# Function to fetch the current day of the week (mocked for testing)
def get_day_of_week():
    return 'Sunday'

# Main function to get user input and print the schedule names and timetable
def main():
    line_id = input("Enter the line ID: ")
    stop_point_id = input("Enter the stop point ID: ")
    direction = input("Enter the direction (outbound/inbound): ")

    # Fetching data
    url = f'https://api.tfl.gov.uk/Line/{line_id}/Timetable/{stop_point_id}?direction={direction}'
    data = fetch_data(url)

    # Extracting schedule names
    schedule_names_dict = extract_schedule_names(data)

    # Fetch day of the week
    day_of_week = get_day_of_week()
    print(f"Today is {day_of_week}.")

    # Print extracted schedule names
    print("\nExtracted Schedule Names:")
    for i, schedule_name in enumerate(schedule_names_dict.keys(), start=1):
        print(f"{i}. {schedule_name}")

    # Determine the matching schedule name based on the current day of the week
    selected_schedule_name = None

    if day_of_week.lower() == 'monday' or day_of_week.lower() == 'tuesday' or day_of_week.lower() == 'wednesday' or day_of_week.lower() == 'thursday':
        # Check for the preferred schedule names for Monday to Thursday
        preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Thursday', 'Monday to Friday']
        for preferred_name in preferred_schedule_names:
            if preferred_name in schedule_names_dict:
                selected_schedule_name = preferred_name
                break
    elif day_of_week.lower() == 'friday':
        # Check for the preferred schedule names for Friday
        preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Friday', 'Friday']
        for preferred_name in preferred_schedule_names:
            if preferred_name in schedule_names_dict:
                selected_schedule_name = preferred_name
                break
    elif day_of_week.lower() == 'saturday':
        # Check for the preferred schedule names for Saturday
        preferred_schedule_names = ['Saturday']
        for preferred_name in preferred_schedule_names:
            if preferred_name in schedule_names_dict:
                selected_schedule_name = preferred_name
                break
    elif day_of_week.lower() == 'sunday':
        # Check for the preferred schedule names for Sunday
        preferred_schedule_names = ['Sunday']
        for preferred_name in preferred_schedule_names:
            if preferred_name in schedule_names_dict:
                selected_schedule_name = preferred_name
                break
    else:
        # For other days of the week, select based on exact match
        for schedule_name in schedule_names_dict.keys():
            if day_of_week.lower() in schedule_name.lower():
                selected_schedule_name = schedule_name
                break

    if selected_schedule_name:
        print(f"\nSelected schedule name: {selected_schedule_name}")

        # Categorizing the timetable into hourly slots
        timetable = schedule_names_dict[selected_schedule_name]
        slots = categorize_into_slots(timetable)

        # Get the current time in BST
        bst = pytz.timezone('Europe/London')
        now = datetime.now(bst)
        current_hour = now.hour
        current_minute = now.minute

        # Calculate the number of buses scheduled between the first time available in the current slot and the current time
        bus_count = calculate_buses(slots[current_hour], current_hour, current_minute)
        print(f"\nNumber of buses scheduled between the first time available in the current slot and now: {bus_count}")

        # Print the timetable slots corresponding to the current hour, the previous hour, and the next hour
        hours_to_print = [current_hour - 1, current_hour, current_hour + 1]
        print("\nTimetable:")
        print("Time   Line ID  Stop Point ID  Direction")
        for hour in hours_to_print:
            if 0 <= hour < 24:
                print(f"\nSlot {hour}:")
                for journey in slots[hour]:
                    journey_hour = str(journey['hour']).zfill(2)
                    journey_minute = str(journey['minute']).zfill(2)
                    time = f"{journey_hour}:{journey_minute}"
                    print(f"{time}   {line_id}    {stop_point_id}  {direction.capitalize()}")

    else:
        print("No matching schedule name found for today.")

# Run the main function
if __name__ == "__main__":
    main()


Enter the line ID: d7
Enter the stop point ID: 490000038F
Enter the direction (outbound/inbound): inbound
Today is Sunday.

Extracted Schedule Names:
1. Monday to Friday
2. Saturday
3. Sunday

Selected schedule name: Sunday

Number of buses scheduled between the first time available in the current slot and now: 2

Timetable:
Time   Line ID  Stop Point ID  Direction

Slot 12:
12:12   d7    490000038F  Inbound
12:27   d7    490000038F  Inbound
12:42   d7    490000038F  Inbound
12:57   d7    490000038F  Inbound

Slot 13:
13:12   d7    490000038F  Inbound
13:27   d7    490000038F  Inbound
13:42   d7    490000038F  Inbound
13:57   d7    490000038F  Inbound

Slot 14:
14:13   d7    490000038F  Inbound
14:28   d7    490000038F  Inbound
14:43   d7    490000038F  Inbound
14:58   d7    490000038F  Inbound
